Import libraries and use Beautiful Soup to get table from web page

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
 
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [2]:
table = soup.find('table', {'class':'wikitable sortable'})

In [3]:
links = table.findAll('a')

Append values from the table into three arrays and create data frame with these arrays

In [4]:
Postal_codes = []
Boroughs = []
Neighbourhoods = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        Postal_codes.append(cells[0].find(text=True))
        Boroughs.append(cells[1].find(text=True))
        Neighbourhoods.append(cells[2].find(text=True))

df = pd.DataFrame()
df['Postal Codes'] = Postal_codes
df['Borough'] = Boroughs
df['Neighbourhood'] = Neighbourhoods

Remove rows where Borough is not assigned

In [5]:
df = df[~df.Borough.str.contains('Not assigned')]

For values in not assigned in Neighbourhood column, assign the same Borough value

In [6]:
for i, j in df.iterrows():
    if j['Neighbourhood'] == 'Not assigned\n':
        j['Neighbourhood'] = j['Borough']


Group all neighbourhoods with same postal code

In [7]:
df = df.groupby(['Postal Codes', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()


Final product

In [13]:
df

,Postal Codes,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae\n
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount\n
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens\n, Ric..."
101,M9V,Etobicoke,"Albion Gardens\n, Beaumond Heights, Humbergate..."


In [14]:
df.shape

(103, 3)